# Meditopia (TR - US) App Store

# Import the Libraries

In [ ]:
!pip install langdetect

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
import math
import seaborn as sns
import nltk
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from langdetect import detect_langs

In [ ]:
nltk.download('punkt')

# Understanding the Data

In [ ]:
df = pd.read_csv("../input/app-store-meditation-app-reviews/appstore_meditopia.csv")
df.head()

In [ ]:
# unnamed is just a duplicate of the index
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# look at the dates separately
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df = df.drop(columns=["date"])
df.head()

In [ ]:
df = df.rename(columns={'developerResponse.body': "meditopia response"})
df.head()

In [ ]:
from nltk import word_tokenize
word_token = [word_tokenize(review) for review in df.review]
len_tokens= [] 

for i in range(len(word_token)):
    len_tokens.append(len(word_token[i]))

df["n_tokens"] = len_tokens
df.head()

## Analyzing MultiLingual Data

In [ ]:
df.tail()

As it can be seen in line 235, there are also some non-English tweets. Let's identify them.

In [ ]:
lang = [detect_langs(i) for i in df.review]

In [ ]:
language = [str(i).split(":")[0][1:] for i in lang]

In [ ]:
df["lang"] = language

In [ ]:
df

There are some errors but removing all non-english tweets will reduce 86 rows.

In [ ]:
(len(df) - len(df[df["lang"] == "en"])) / len(df)

In [ ]:
fix = df[df["lang"] != "en"]

In [ ]:
fix.rating.mean()

In [ ]:
# correcting some wrong lang classification
df.loc[89, "lang"] = "en"
df.loc[90, "lang"] = "en"
df.loc[131, "lang"] = "en"
df.loc[132, "lang"] = "en"
df.loc[141, "lang"] = "en"

In [ ]:
df.groupby(by="lang").mean()

### Correlation

In [ ]:
# see if there is a correlation on the numeric data
correlation = df.corr()
correlation

In [ ]:
df["year"].min()

In [ ]:
df["year"].max()

In [ ]:
# visualize (there is no correlation)
sns.heatmap(correlation)
plt.show()

## Data Visualization

In [ ]:
df.info()

In [ ]:
sns.catplot(x="year",data=df,kind="count")
plt.show()

In [ ]:
sns.countplot(x="year", data=df, hue="rating");

In [ ]:
df.columns

In [ ]:
sns.catplot(x="rating",data=df,kind="count")
plt.show()

In [ ]:
sns.catplot(x="month",data=df,kind="count")
plt.show()

# Analysing Reviews

In [ ]:
df['review'] = df['review'].str.replace('[^\w\s]','')
df['review'] = df['review'].str.lower()
df.head()

In [ ]:
df.isna().sum()

Calm hasn't responded some of the reviews. Check if there is a relation between comments Calm answered and ratings.

## Word Cloud

I will only check English tweets.

In [ ]:
df_1 = df[(df["rating"] == 1) & (df["lang"] == "en")]
df_2 = df[(df["rating"] == 2) & (df["lang"] == "en")]
df_3 = df[(df["rating"] == 3) & (df["lang"] == "en")]
df_4 = df[(df["rating"] == 4) & (df["lang"] == "en")]
df_5 = df[(df["rating"] == 5) & (df["lang"] == "en")]

In [ ]:
text_1 = " ".join(review for review in df_1.review)
text_2 = " ".join(review for review in df_2.review)
text_3 = " ".join(review for review in df_3.review)
text_4 = " ".join(review for review in df_4.review)
text_5 = " ".join(review for review in df_5.review)

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["meditation", "app", "meditate", "im", "meditations" ,"thing"])

cloud_1 = WordCloud(stopwords=stopwords, background_color="white").generate(text_1)
cloud_2 = WordCloud(stopwords=stopwords, background_color="white").generate(text_2)
cloud_3 = WordCloud(stopwords=stopwords, background_color="white").generate(text_3)
cloud_4 = WordCloud(stopwords=stopwords, background_color="white").generate(text_4)
cloud_5 = WordCloud(stopwords=stopwords, background_color="white").generate(text_5)

In [ ]:
width=5
height=5
rows = 1
cols = 5
axes=[]
fig=plt.figure(figsize=(40, 40))

for a in range(rows*cols):
    cloud = [cloud_1, cloud_2, cloud_3, cloud_4, cloud_5]
    axes.append(fig.add_subplot(rows, cols, a+1) )
    subplot_title=("Word Cloud"+str(a + 1))
    axes[-1].set_title(subplot_title)  
    plt.imshow(cloud[a])
fig.tight_layout()    
plt.show()

## Implementing TF-IDF

In [ ]:
vect = TfidfVectorizer(max_features = 10, stop_words=stopwords)
tfIdf = vect.fit(df_1.review)
X = vect.transform(df_1.review)
X_df = pd.DataFrame(X.toarray(), columns = vect.get_feature_names())
X_df.head()

In [ ]:
len(df_1.index)

In [ ]:
stopwords.update(["really"])
vect = TfidfVectorizer(max_features = 15, stop_words=stopwords)
tfIdf = vect.fit(df_2.review)
X = vect.transform(df_2.review)
X_df = pd.DataFrame(X.toarray(), columns = vect.get_feature_names())
X_df.head()

In [ ]:
len(df_2.index)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
print(df_2[df_2['review'].str.contains("press")]["review"])

In [ ]:
len(df_3.index)

In [ ]:
vect = TfidfVectorizer(max_features = 15, stop_words=stopwords)
tfIdf = vect.fit(df_3.review)
X = vect.transform(df_3.review)
X_df = pd.DataFrame(X.toarray(), columns = vect.get_feature_names())
X_df.head()

In [ ]:
len(df_4.index)

In [ ]:
vect = TfidfVectorizer(max_features = 15, stop_words=stopwords)
tfIdf = vect.fit(df_4.review)
X = vect.transform(df_4.review)
X_df = pd.DataFrame(X.toarray(), columns = vect.get_feature_names())
X_df.head()

In [ ]:
len(df_5.index)

In [ ]:
vect = TfidfVectorizer(max_features = 15, stop_words=stopwords)
tfIdf = vect.fit(df_5.review)
X = vect.transform(df_5.review)
X_df = pd.DataFrame(X.toarray(), columns = vect.get_feature_names())
X_df.head()

# Sentiment Analysis

In [ ]:
from textblob import TextBlob 

In [ ]:
sentiment = []

for i in df.review:
    text = TextBlob(i)
    sentiment.append(text.sentiment)


In [ ]:
sentiment[2]

In [ ]:
sent = [float(str(i).split(",")[0].split("=")[1]) * 100 for i in sentiment]

In [ ]:
df["sentiment"] = sent

In [ ]:
df.head(2)

In [ ]:
sns.relplot(x="year", y="sentiment", kind="line", data=df, ci=None);

In [ ]:
sns.relplot(x="rating", y="sentiment", kind="scatter", hue="year", data=df);

## Bag of Words

In [ ]:
my_stopwords = ENGLISH_STOP_WORDS.union(["meditation", "app", "meditate", "im", "meditations" ,"thing"])

### Rating 1

In [ ]:
len(df_1["review"])

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,3), max_features = 100, max_df=500, stop_words=my_stopwords)
vectorizer.fit(df_1.review)
X = vectorizer.transform(df_1.review)

In [ ]:
"""
Sparse matrix is a matrix which 
contains very few non-zero elements. When a sparse matrix is represented with a 2-dimensional array, 
we waste a lot of space to represent that matrix.
"""
X

In [ ]:
X_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names() )

In [ ]:
X_df.head()

In [ ]:
X_df.columns

In [ ]:
print(df_1[df_1['review'].str.contains("miss")]["review"])

### Rating 2

In [ ]:
len(df_2.index)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2), max_features = 100, max_df=500, stop_words=my_stopwords)
vectorizer.fit(df_2.review)
X = vectorizer.transform(df_2.review)
X_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names())
X_df.head()

In [ ]:
X_df.columns

In [ ]:
print(df_2["review"])

### Rating 3

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2), max_features = 100, max_df=500, stop_words=my_stopwords)
vectorizer.fit(df_3.review)
X = vectorizer.transform(df_3.review)
X_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names())
X_df.head()

In [ ]:
X_df.columns

In [ ]:
print(df_3["review"])

### Rating 4

In [ ]:
vectorizer = CountVectorizer(max_features = 100, max_df=500, stop_words=my_stopwords)
vectorizer.fit(df_4.review)
X = vectorizer.transform(df_4.review)
X_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names())
X_df.head()

In [ ]:
X_df.columns

In [ ]:
df_4

### Rating 5

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1,2), max_features = 100, max_df=500, stop_words=my_stopwords)
vectorizer.fit(df_5.review)
X = vectorizer.transform(df_5.review)
X_df = pd.DataFrame(X.toarray(), columns= vectorizer.get_feature_names())
X_df.head()

In [ ]:
X_df.columns

In [ ]:
print(df_5[df_5['review'].str.contains("meditopia")]["review"])

## Stemming and Lemmatization

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [ ]:
porter = PorterStemmer()
WNLemmatizer = WordNetLemmatizer()
tokens = [word_tokenize(text) for text in df.review]

In [ ]:
stem = [porter.stem(str(text)) for text in tokens]

In [ ]:
nltk.download('wordnet')

In [ ]:
lem = [WNLemmatizer.lemmatize(str(text)) for text in tokens]

In [ ]:
stem[0]

In [ ]:
lem[0]

In [ ]:
list(set(lem) - set(stem))

## Looking at Turkish Data

In [ ]:
df[df["lang"] == "tr"]

In [ ]:
df.to_csv('meditopia_final_app.csv', index=False) 

In [ ]:
print(df[df['review'].str.contains("student")]["review"])